# Productionazing Machine Learning with Vertex AI

### Set up project ID

In [248]:
import os

PROJECT_ID = "crazy-hippo-01"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  crazy-hippo-01


### Define Current Timestamp

In [249]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
print('Timestamp: ', TIMESTAMP)

Timestamp:  20210524115654


### Create storage bucket

In [250]:
BUCKET_NAME = "gs://crazy-vertex-ai-pipelines"  # @param {type:"string"} 
REGION = "us-central1"  # @param {type:"string"}

In [34]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "-vertex-" + TIMESTAMP

In [35]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://crazy-vertex-ai-pipelines/...


In [59]:
! gsutil ls -al $BUCKET_NAME

### Import Libraries and define constants

In [251]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

USER = "crazy-hippo"  # <---CHANGE THIS
PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)

PIPELINE_ROOT

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin


'gs://crazy-vertex-ai-pipelines/pipeline_root/crazy-hippo'

In [252]:
import json
from typing import NamedTuple

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google.client import AIPlatformClient

from kfp.v2.dsl import (
    Input,
    Output,
    Artifact,
    Dataset,
    Metrics
)

### Define Pipeline Comnponents

#### Pipeline Step 1 - Extract and Transform Data

In [240]:
@component(output_component_file='read_transform_data.yaml',
          base_image='python:3.9',
          packages_to_install=['pandas', 
                             'google-cloud-bigquery', 
                             'pyarrow' , 
                             'gcsfs', 
                             'sklearn'])
def read_transform_data() -> NamedTuple(
      'ComponentOutputs',
      [
        ('training_data', str),
        ('test_data', str),
        ('validation_data', str),
      ]):
    
    #Import libraries
    import pandas as pd
    import time
    from sklearn.model_selection import train_test_split
    from google.cloud.bigquery import Client, QueryJobConfig
    
    #Initiate BigQuery Client
    client = Client(project='crazy-hippo-01')
    
    query = """SELECT age, workclass, occupation, education_num, marital_status, capital_gain, income_bracket
    FROM `crazy-hippo-01.census_data_us.census_raw` 
    """
    #Run Query
    job = client.query(query)
    #retry=bq_retry.DEFAULT_RETRY
    df = job.to_dataframe()
    
    #df = pd.read_csv('gs://crazy-hippo-01/dataset/census_train.csv')
    
    #df = df[['age', 'workclass', 'occupation', 'education_num', 'marital_status', 'capital_gain', 'income_bracket']]
    
    #Drop null values in dataset
    df = df.dropna()
    
    #Create training, test and validation datasets
    train, test = train_test_split(df, test_size=0.20, random_state=42)
    train, val = train_test_split(train, test_size=0.20, random_state=42)

    #Define Staging Bucket in GCS
    BUCKET = 'gcs://crazy-hippo-01/kubeflow_staging/'
    
    #Define Datasets Names
    TRAIN_DATA = BUCKET + 'datasets/training/training{}'.format(str(int(time.time())))  + '.csv'
    TEST_DATA = BUCKET + 'datasets/testing/test{}'.format(str(int(time.time())))  + '.csv'
    VALIDATION_DATA = BUCKET + 'datasets/validation/validation{}'.format(str(int(time.time())))  + '.csv'

    #Write data to GCS Storage
    train.to_csv(TRAIN_DATA, index=False, header=True)
    test.to_csv(TEST_DATA, index=False, header=True)
    val.to_csv(VALIDATION_DATA, index=False, header=True)

    #Define outputs with namedtuple
    from collections import namedtuple
    
    return_values = namedtuple(
      'ComponentOutputs',
        ['training_data', 'test_data', 'validation_data'])
        
    return return_values(TRAIN_DATA, TEST_DATA, VALIDATION_DATA)


#### Pipeline Step 2 - Train and Save Model

In [241]:
@component(output_component_file='train_model.yaml',
          base_image='python:3.9',
          packages_to_install=['pandas', 
                             'pyarrow' , 
                             'gcsfs' , 
                             'google-cloud-bigquery-storage',
                             'tensorflow'])
def train_model(TRAIN_DATA:str, TEST_DATA:str, VALIDATION_DATA:str) -> NamedTuple(
      'ComponentOutputs',
      [
        ('model_path', str)
      ]):
    
    import pandas as pd
    import time
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.layers.experimental import preprocessing
    
    #VARIABLES AND TRAINING PARAMETERS
    TRAINING_DATA = pd.read_csv(TRAIN_DATA)
    TESTING_DATA = pd.read_csv(TEST_DATA)
    VALIDATION_DATA = pd.read_csv(VALIDATION_DATA)
    
    BATCH_SIZE = 32
    
    print(tf.__version__)
    
    print(TRAINING_DATA.head())
    
    print(TESTING_DATA.head())
    
    print(VALIDATION_DATA.head())
    
    #TENSORFLOW DATASET FUNCTION
    def helperfunc_create_dataset(dataframe, shuffle=True, batch_size=5):
        dataframe = dataframe.copy()
        labels = dataframe.pop('income_bracket')
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
        if shuffle:
            ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
        ds = ds.prefetch(batch_size)
        return ds
    
    #NORMALIZATION FUNCTION
    def helperfunc_get_normalization_layer(name, dataset):
        # Create a Normalization layer for our feature.
        normalizer = preprocessing.Normalization()

        # Prepare a Dataset that only yields our feature.
        feature_ds = dataset.map(lambda x, y: x[name])

        # Learn the statistics of the data.
        normalizer.adapt(feature_ds)

        return normalizer
    
    #CATEGORY ENCODING FUNCTION
    def helperfunc_get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
        # Create a StringLookup layer which will turn strings into integer indices
        if dtype == 'string':
            index = preprocessing.StringLookup(max_tokens=max_tokens)
        else:
            index = preprocessing.IntegerLookup(max_values=max_tokens)

        # Prepare a Dataset that only yields our feature
        feature_ds = dataset.map(lambda x, y: x[name])

        # Learn the set of possible values and assign them a fixed integer index.
        index.adapt(feature_ds)

        # Create a Discretization for our integer indices.
        encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

        # Prepare a Dataset that only yields our feature.
        feature_ds = feature_ds.map(index)

        # Learn the space of possible indices.
        encoder.adapt(feature_ds)

        # Apply one-hot encoding to our indices. The lambda function captures the
        # layer so we can use them, or include them in the functional model later.
        return lambda feature: encoder(index(feature))
    
    #CREATE TENSORFLOW DATASETS
    TRAIN_DS = helperfunc_create_dataset(TRAINING_DATA, batch_size=BATCH_SIZE)
    VALIDATION_DS = helperfunc_create_dataset(VALIDATION_DATA, shuffle=False, batch_size=BATCH_SIZE)
    TESTING_DS = helperfunc_create_dataset(TESTING_DATA, shuffle=False, batch_size=BATCH_SIZE)
    
    #CREATE PREPROCESSING LAYERS
    ALL_INPUTS = []
    ENCODED_FEATURES = []

    NUMERICAL = ['age' , 'capital_gain']
    CATEGORICAL_INT_COLS = ['education_num']
    CATEGORICAL_STRING_COLS = ['occupation', 
                               'workclass', 
                               'marital_status']
    TARGET = ['income_bracket']
    
    # Numeric features.
    for header in NUMERICAL:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = helperfunc_get_normalization_layer(header, TRAIN_DS)
        encoded_numeric_col = normalization_layer(numeric_col)
        ALL_INPUTS.append(numeric_col)
        ENCODED_FEATURES.append(encoded_numeric_col)
        
    # Categorical features encoded as integers.
    for header in CATEGORICAL_INT_COLS:
        categorical_int_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
        encoding_layer = helperfunc_get_category_encoding_layer(header, TRAIN_DS, dtype='int64', max_tokens=5)
        encoded_categorical_int_col = encoding_layer(categorical_int_col)
        ALL_INPUTS.append(categorical_int_col)
        ENCODED_FEATURES.append(encoded_categorical_int_col)
    
    # Categorical features encoded as string.
    for header in CATEGORICAL_STRING_COLS:
        categorical_string_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
        encoding_layer = helperfunc_get_category_encoding_layer(header, TRAIN_DS, dtype='string', max_tokens=5)
        encoded_categorical_string_col = encoding_layer(categorical_string_col)
        ALL_INPUTS.append(categorical_string_col)
        ENCODED_FEATURES.append(encoded_categorical_string_col)
    
        
    #CREATE and COMPILE MODEL
    all_features = tf.keras.layers.concatenate(ENCODED_FEATURES)
    x = tf.keras.layers.Dense(32, activation="relu")(all_features)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(ALL_INPUTS, output)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=["accuracy"])
    
    #TRAIN MODEL
    history = model.fit(TRAIN_DS, epochs=10, validation_data=VALIDATION_DS)
    
    
    #Define Bucket in GCS for Model Storage
    BUCKET = 'gs://crazy-hippo-01/kubeflow_staging/models/'
    
    #Define Datasets Names
    MODEL_PATH = BUCKET + 'earnings_model{}'.format(str(int(time.time())))
    
    #Save model to Artifact Store for Project
    model.save(MODEL_PATH)
    
    print('Model saved to: ' + MODEL_PATH)
    
    #Define outputs with namedtuple
    from collections import namedtuple
    
    return_values = namedtuple(
      'ComponentOutputs',
        ['model_path'])
        
    return return_values(MODEL_PATH)

#### Pipeline Step 3 - Evaluate Saved Model

In [242]:
@component(output_component_file='evaluate_model.yaml',
          base_image='python:3.9',
          packages_to_install=['pandas',
                         'google-cloud-bigquery',
                         'pyarrow', 
                         'gcsfs',
                         'tensorflow'])
def evaluate_validate_model(saved_model:str, test_dataset:str, pipeline:str, framework:str, metrics: Output[Metrics]) -> NamedTuple(
      'ComponentOutputs',
      [
          ('accuracy', float),
          ('loss', float),
      ]):
    
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    
    print(saved_model, test_dataset, pipeline, framework)
    
    #HELPER FUNCTION - TENSORFLOW DATASET FUNCTION
    def helperfunc_create_dataset(dataframe, shuffle=True, batch_size=5):
        dataframe = dataframe.copy()
        labels = dataframe.pop('income_bracket')
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
        if shuffle:
            ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
        ds = ds.prefetch(batch_size)
        return ds
    
    #LOAD TRAINED MODEL FROM ARTIFACT STORE
    reloaded_model = tf.keras.models.load_model(saved_model)
    
    #READ TESTING DATASET
    TESTING_DATA = pd.read_csv(test_dataset)

    #SET BATCG SIZE
    BATCH_SIZE = 32
    
    #CALL HELPER FUNCTION TO CREATE TENSORFLOW DATASET
    TESTING_DS = helperfunc_create_dataset(TESTING_DATA, shuffle=False, batch_size=BATCH_SIZE)
    
    #EVALUATE MODEL WITH TEST DATA
    loss, accuracy = reloaded_model.evaluate(TESTING_DS)
    
    accuracy = float(accuracy)
    loss = float(loss)
    
    #PRINT ACCURACY METRIC
    print("Accuracy", accuracy)
    print("Loss", loss)
    
    
    from tensorflow.python.lib.io import file_io    
    
    #Write Metrics to BigQuery Table for Validation and possible promotion to Deployment
    from google.cloud.bigquery import Client, QueryJobConfig
    
    #Initiate BigQuery Client
    client = Client(project='crazy-hippo-01')
    
    print('Sending Metrics into BigQuery')
    
    #Define DML Query to Insert Metrics into BugQuery
    query = """INSERT `crazy-hippo-01.census_data_us.model_metrics_history` (model_name, pipeline, framework, accuracy, loss)
    VALUES ("{}", "{}", "{}", {}, {})  
    """.format(saved_model, pipeline, framework, accuracy, loss)
    
    #Run Query
    job = client.query(query)
    
    print('Metrics sent to BigQuery!')
    
    # Export two metrics
    metrics.log_metric('accuracy', accuracy)
    metrics.log_metric('loss', loss)

    from collections import namedtuple
    
    component_outputs = namedtuple('ComponentOutputs',
        ['accuracy', 'loss'])
        
    return component_outputs(accuracy, loss)

### Define Pipeline

In [243]:
@dsl.pipeline(
  name='binaryclassmodel3',
  description='Binary Classification Model with Tensorflow Deep Learning and Connected Pre-processing Layers'
)
def binary_classifier_earnings_v3(
    pipeline: str = 'Tensorflow DL model with Integrated Preprocessing Version 1',
    framework: str = 'Tensorflow'
    ):
   
    first_step = read_transform_data()
    #first_step.execution_options.caching_strategy.max_cache_staleness = "P0D"

   
    second_step = train_model(first_step.outputs['training_data'], 
                             first_step.outputs['test_data'], 
                             first_step.outputs['validation_data'])
    #second_step.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    
    third_step = evaluate_validate_model(second_step.outputs['model_path'], 
                                        first_step.outputs['test_data'],
                                       pipeline, framework)
    #third_step.execution_options.caching_strategy.max_cache_staleness = "P0D"

### Compile and run Pipeline

Compile pipeline

In [244]:
from kfp.v2 import compiler  

compiler.Compiler().compile(
    pipeline_func=binary_classifier_earnings_v3, package_path="earnings_pipeline_ver3.json"
)

Instantiate the API client object

In [245]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

Run pipeline

In [246]:
SERVICE_ACCOUNT = 'pipelines-vertex-ai@crazy-hippo-01.iam.gserviceaccount.com'

In [247]:
response = api_client.create_run_from_job_spec(
    job_spec_path="earnings_pipeline_ver3.json",
    pipeline_root=PIPELINE_ROOT,
    service_account=SERVICE_ACCOUNT 
)

Alternate Test SA

In [ ]:
SERVICE_ACCOUNT = 'owner-sa@crazy-hippo-01.iam.gserviceaccount.com'